In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

### Load Coordinates Data

In [2]:
#load canada data from CSV
df=pd.read_csv('canada_cd.csv',index_col=0)
print('Data loaded')

Data loaded


In [3]:
neighborhoods_data = df.drop(['Address'],axis=1)
print('Neighbourhood Data')

Neighbourhood Data


In [4]:
#df.loc[:, ~df.columns.str.contains('^Unnamed')]
neighborhoods_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.653226,-79.383184
1,M4A,North York,Victoria Village,43.653226,-79.383184
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.653226,-79.383184
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.653226,-79.383184
4,M7A,Queen's Park,Queen's Park,43.653226,-79.383184


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods_data['Borough'].unique()),
        neighborhoods_data.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto, Ontario.

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### Create a map of Toronto, Ontario with neighborhoods superimposed on top.

In [7]:
# create map of New York using latitude and longitude values
map_Ontario = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_data['Latitude'], neighborhoods_data['Longitude'], neighborhoods_data['Borough'], neighborhoods_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Ontario)  
    
map_Ontario

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data

In [8]:
toronto_data = neighborhoods_data[neighborhoods_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.653226,-79.383184
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.653226,-79.383184
2,M5C,Downtown Toronto,St. James Town,43.653226,-79.383184
3,M4E,East Toronto,The Beaches,43.653226,-79.383184
4,M5E,Downtown Toronto,Berczy Park,43.653226,-79.383184


In [9]:
toronto_data.shape

(38, 5)

### Now lets visualize toronto the neighborhoods in it.

In [10]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

###  Next, I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [39]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'ID' # your Foursquare ID
CLIENT_SECRET = 'Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ID
CLIENT_SECRET:Secret


#### Let's explore the first neighborhood in our dataframe.

In [13]:
toronto_data.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

Get the neighborhood's latitude and longitude values.

In [14]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.653226000000004, -79.3831843.


#### Now, let's get the top 100 venues that are in Harbourfront,Regent Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [15]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=BV1LXDWAZQ5VQM130IOGXBWOBK02BUAIBJHBQUG03ZOSTQTJ&client_secret=J5KFKPP0UCNPIMJ5UZV41S53CHUSUPY14HJNVPGD4UHTHEGG&v=20180605&ll=43.653226000000004,-79.3831843&radius=500&limit=100'

Send a GET request

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d11021b018cbb002cb83be4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 136,
  'suggestedBounds': {'ne': {'lat': 43.657726004500006,
    'lng': -79.37697640702638},
   'sw': {'lat': 43.6487259955, 'lng': -79.38939219297362}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
     

Now using the get_category_type function from the Foursquare lab.

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

we are ready to clean the json and structure it into a pandas dataframe.

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Indigo,Bookstore,43.653515,-79.380696
4,CF Toronto Eaton Centre,Shopping Mall,43.653852,-79.380655


The number of venues returned by Foursquare is given below

In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The next code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [21]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


#### Examining the size of the resulting dataframe

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(3800, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.653226,-79.383184,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,"Harbourfront,Regent Park",43.653226,-79.383184,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Harbourfront,Regent Park",43.653226,-79.383184,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
3,"Harbourfront,Regent Park",43.653226,-79.383184,Indigo,43.653515,-79.380696,Bookstore
4,"Harbourfront,Regent Park",43.653226,-79.383184,CF Toronto Eaton Centre,43.653852,-79.380655,Shopping Mall


Let's see how many venues were returned for each neighborhood

In [23]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton,Exhibition Place,Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",100,100,100,100,100,100
"Cabbagetown,St. James Town",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


#### checking the unique categories returned

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 64 uniques categories.


#### Now, we are ready to analyze Each Neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Lingerie Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Office,Opera House,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Women's Store
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [26]:
toronto_onehot.shape

(3800, 65)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Lingerie Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Office,Opera House,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Women's Store
0,"Adelaide,King,Richmond",0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
1,Berczy Park,0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
2,"Brockton,Exhibition Place,Parkdale Village",0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
3,Business Reply Mail Processing Centre 969 Eastern,0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
5,"Cabbagetown,St. James Town",0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
6,Central Bay Street,0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
7,"Chinatown,Grange Park,Kensington Market",0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.03,0.02,0.01,0.02,0.01
8,Christie,0.01,0.04,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.02,0.02,0.05,0.07,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0

##### Let's confirm the new size

In [28]:
toronto_grouped.shape

(38, 65)

#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.07
1       Clothing Store  0.05
2       Cosmetics Shop  0.04
3  American Restaurant  0.04
4           Restaurant  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.07
1       Clothing Store  0.05
2       Cosmetics Shop  0.04
3  American Restaurant  0.04
4           Restaurant  0.03


----Brockton,Exhibition Place,Parkdale Village----
                 venue  freq
0          Coffee Shop  0.07
1       Clothing Store  0.05
2       Cosmetics Shop  0.04
3  American Restaurant  0.04
4           Restaurant  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                 venue  freq
0          Coffee Shop  0.07
1       Clothing Store  0.05
2       Cosmetics Shop  0.04
3  American Restaurant  0.04
4           Restaurant  0.03


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue  fre

#### Let's put that into a *pandas* dataframe

First, let's use the function from the lab to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now, we can create a dataframe for the top 10 venues for each Neighbourhood

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
1,Berczy Park,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
5,"Cabbagetown,St. James Town",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
6,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
7,"Chinatown,Grange Park,Kensington Market",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
8,Christie,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
9,Church and Wellesley,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

 creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.653226,-79.383184,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.653226,-79.383184,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
2,M5C,Downtown Toronto,St. James Town,43.653226,-79.383184,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
3,M4E,East Toronto,The Beaches,43.653226,-79.383184,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
4,M5E,Downtown Toronto,Berczy Park,43.653226,-79.383184,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater


Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
3,East Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
5,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
6,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
7,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
8,West Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Steakhouse,Breakfast Spot,Restaurant,Tea Room,Plaza,Theater


#### Cluster 2

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 3

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Gastropub,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bakery
7,Downtown Toronto,4,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Restaurant,Gym,Burger Joint,Hotel
8,West Toronto,4,Bakery,Bar,Café,Coffee Shop,Market,Fast Food Restaurant,Japanese Restaurant,Farmers Market,Beer Store,Bridal Shop
13,West Toronto,4,Bar,Coffee Shop,Grocery Store,Italian Restaurant,Café,Deli / Bodega,Arts & Crafts Store,Electronics Store,Hardware Store,Yoga Studio
15,Downtown Toronto,4,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Italian Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Steakhouse
17,Central Toronto,4,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bakery,Pub,Burger Joint,Asian Restaurant,Bank,Tea Room,Bus Line
21,West Toronto,4,Café,Mexican Restaurant,Bar,Furniture / Home Store,Bookstore,Diner,Fast Food Restaurant,Italian Restaurant,Flea Market,Bakery
23,Central Toronto,4,Grocery Store,Pizza Place,Ice Cream Shop,Dessert Shop,Yoga Studio,Massage Studio,Sandwich Place,Pharmacy,Park,Clothing Store
26,Downtown Toronto,4,Café,Coffee Shop,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Restaurant,Massage Studio,Video Game Store
29,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Bakery,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Coffee Shop,Ice Cream Shop
